# ONE-DATA-STUDIO 基础对话示例

本 Notebook 演示如何使用 LangChain 与 ONE-DATA-STUDIO (Bisheng) 的 LLM 服务进行交互。

## 前置条件

1. ONE-DATA-STUDIO 服务已启动
2. 已部署可用的 LLM 模型
3. 安装必要的 Python 包

In [ ]:
# 安装依赖
# !pip install langchain langchain-core requests

## 1. 配置连接

In [ ]:
import os
import sys

# 添加父目录到路径
sys.path.insert(0, os.path.dirname(os.getcwd()))

# 配置 API 连接
BISHENG_API_BASE = os.getenv("BISHENG_API_BASE", "http://localhost:8000")
BISHENG_API_KEY = os.getenv("BISHENG_API_KEY", "")
MODEL_NAME = os.getenv("MODEL_NAME", "qwen-7b-chat")

print(f"API Base: {BISHENG_API_BASE}")
print(f"Model: {MODEL_NAME}")

## 2. 创建 LLM 实例

In [ ]:
from bisheng_llm import BishengLLM

# 创建 LLM 实例
llm = BishengLLM(
    api_base=BISHENG_API_BASE,
    model_name=MODEL_NAME,
    api_key=BISHENG_API_KEY,
    temperature=0.7,
    max_tokens=1024,
)

print(f"LLM Type: {llm._llm_type}")
print(f"Identifying Params: {llm._identifying_params}")

## 3. 基础调用

In [ ]:
# 简单问答
question = "什么是机器学习？请用简单的语言解释。"
response = llm.invoke(question)
print(f"问题: {question}")
print(f"\n回答: {response}")

## 4. 使用 PromptTemplate

In [ ]:
from langchain.prompts import PromptTemplate

# 创建 Prompt 模板
template = """你是一位专业的技术专家。请回答以下关于 {topic} 的问题：

问题：{question}

请提供详细且易于理解的回答："""

prompt = PromptTemplate(
    input_variables=["topic", "question"],
    template=template
)

# 格式化 Prompt
formatted_prompt = prompt.format(
    topic="Python 编程",
    question="如何高效地处理大文件？"
)

print("Formatted Prompt:")
print(formatted_prompt)
print("\n" + "="*50 + "\n")

# 调用 LLM
response = llm.invoke(formatted_prompt)
print("Response:")
print(response)

## 5. 使用 LCEL (LangChain Expression Language)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# 创建链
chain = prompt | llm | StrOutputParser()

# 执行链
result = chain.invoke({
    "topic": "数据库优化",
    "question": "如何优化 SQL 查询性能？"
})

print("Chain Result:")
print(result)

## 6. 多轮对话

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# 创建对话记忆
memory = ConversationBufferMemory()

# 创建对话链
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [ ]:
# 第一轮对话
response1 = conversation.predict(input="你好！我想学习 Python。")
print(f"AI: {response1}")

In [ ]:
# 第二轮对话 (会记住上下文)
response2 = conversation.predict(input="我应该从哪里开始？")
print(f"AI: {response2}")

In [ ]:
# 第三轮对话
response3 = conversation.predict(input="有哪些推荐的学习资源？")
print(f"AI: {response3}")

In [ ]:
# 查看对话历史
print("\n对话历史:")
print(memory.buffer)

## 7. 流式输出

In [ ]:
# 创建支持流式的 LLM
streaming_llm = BishengLLM(
    api_base=BISHENG_API_BASE,
    model_name=MODEL_NAME,
    api_key=BISHENG_API_KEY,
    streaming=True,
)

# 流式输出
print("流式输出演示:")
for chunk in streaming_llm.stream("请写一首关于春天的短诗。"):
    print(chunk, end="", flush=True)
print()  # 换行

## 8. 批量处理

In [ ]:
# 批量问题
questions = [
    "Python 的优点是什么？",
    "JavaScript 和 Python 的区别？",
    "如何选择编程语言？"
]

# 批量调用
responses = llm.batch(questions)

for q, r in zip(questions, responses):
    print(f"Q: {q}")
    print(f"A: {r[:200]}...\n")  # 只显示前200字符

## 总结

本 Notebook 演示了：

1. **基础配置** - 如何配置 Bisheng LLM 连接
2. **简单调用** - 直接调用 LLM 进行问答
3. **Prompt 模板** - 使用 LangChain PromptTemplate 格式化输入
4. **LCEL 链** - 使用 LangChain Expression Language 构建处理链
5. **多轮对话** - 使用 Memory 实现上下文记忆
6. **流式输出** - 实时获取生成结果
7. **批量处理** - 同时处理多个请求

更多高级用法请参考 RAG Pipeline 示例。